<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%%capture
!pip install transformers
!pip install python-chess

In [28]:
from datetime import datetime

from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data_with_buckets.csv"
csv_path = f"{folder}/{csv_file}"


now = datetime.now()
dt_string = now.strftime("%m_%d_%Y_%H_%M")

model_file = "classifier_model.pth"
datetime = dt_string
model_path = f"{folder}/{datetime}_{model_file}"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from imblearn.under_sampling import RandomUnderSampler

###Parameters

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

params = {
    "batch_size": 4,
    "epochs": 5,
    "learning_rate": 0.00003,
    "warmup_steps": 0.01,
    "epsilon": 1e-8,
    "accum_iter": 8,
    "num_classes": 6,
    "dropout": 0.5
}

class Params:
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

params = Params(**params)

### Classify Model

In [31]:
class BertClassifier(nn.Module):
  def __init__(self, dropout=params.dropout, num_classes=params.num_classes):
    super(BertClassifier, self).__init__()

    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768, num_classes)
    # self.relu = nn.ReLU()
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)
    final_layer = self.softmax(linear_output)

    return final_layer

### Dataset Class

In [32]:
class ChessDataset(torch.utils.data.Dataset):
  def __init__(self, moves, labels):
      tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
      self.labels = torch.Tensor(labels.values)
      self.moves = [tokenizer(move, 
                              padding='max_length', max_length = 256, truncation=True,
                              return_tensors="pt") for move in moves]

  def __len__(self):
      return len(self.labels)

  def __getitem__(self, idx):
      return self.moves[idx], self.labels[idx]

### Training Function

In [41]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer):
    num_batches = len(train_dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = params.warmup_steps, num_training_steps = num_batches * params.epochs)
    print(f"Number of batches: {num_batches}")
    for epoch_num in range(params.epochs):
        total_acc_train = 0
        total_loss_train = 0

        for batch_idx, (train_input, train_label) in enumerate(train_dataloader):
        #   print(sum(train_input['input_ids'].squeeze(1)[0] == 0))
          train_label = train_label.to(device)
          mask = train_input['attention_mask'].to(device)
          input_id = train_input['input_ids'].squeeze(1).to(device)

          output = model(input_id, mask)
          
          batch_loss = criterion(output, train_label.long())
          total_loss_train += batch_loss.item()
          
          acc = (output.argmax(dim=1) == train_label).sum().item()
          total_acc_train += acc

          batch_loss.backward()

          # Gradient Accumulation
          if ((batch_idx + 1) % params.accum_iter == 0) or (batch_idx + 1 == num_batches):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

          if batch_idx % 50 == 0:
            print(f"Batch Number: {batch_idx}")
            print(f"Model Output: {torch.exp(output).tolist()}")
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

          for val_input, val_label in val_dataloader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, val_label.long())
            total_loss_val += batch_loss.item()
            
            acc = (output.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataloader): .3f} \
            | Train Accuracy: {total_acc_train / len(train_dataloader): .3f} \
            | Val Loss: {total_loss_val / len(val_dataloader): .3f} \
            | Val Accuracy: {total_acc_val / len(val_dataloader): .3f}')
        
        torch.save(model.state_dict(), model_path)

### Test Function

In [46]:
def test(model, test_dataloader):
  total_acc_test = 0
  with torch.no_grad():
    for test_input, test_label in test_dataloader:
      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)
      print(f"Model Output: {torch.exp(output).tolist()}")
      print(f"Prediction: {output.argmax(dim=1).tolist()}, Truth: {test_label.tolist()}")

      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc
       
      print(f"Test Accuracy: {total_acc_test / len(test_dataloader): .3f}")

### Loading Data

In [35]:
%%capture
chess_data = pd.read_csv(csv_path)[0:3000]
X = chess_data["Moves"]
y = chess_data["Bucket"]

balancer = RandomUnderSampler()
X = X.values.reshape(-1, 1)
X, y = balancer.fit_resample(X, y)
rating_counts = y.value_counts().sort_index()
X = X.flatten()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, shuffle=True) # Splits into train/test
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, shuffle=True) # Splits train into train/val

train_data, val_data, test_data = ChessDataset(X_train, y_train), ChessDataset(X_val, y_val), ChessDataset(X_test, y_test)

In [42]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=params.batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=params.batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data)

### Initialize Model

In [55]:
%%capture
model = BertClassifier()
model = model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr = params.learning_rate)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Run Training

In [56]:
train(model, train_dataloader, val_dataloader, criterion, optimizer)

Number of batches: 175
Batch Number: 0
Model Output: [[-2.1017754077911377, -2.5098466873168945, -2.672037124633789, -2.0587315559387207, -0.8309098482131958, -1.8072774410247803], [-1.659617304801941, -3.537363052368164, -3.015733242034912, -1.1911700963974, -2.2322874069213867, -1.137747883796692], [-1.4947092533111572, -2.511418342590332, -2.658843517303467, -1.6807351112365723, -1.2118192911148071, -1.9617536067962646], [-1.9843368530273438, -2.668450117111206, -2.550992727279663, -0.8165637850761414, -1.7257781028747559, -2.3519623279571533]]
Batch Number: 50
Model Output: [[-1.941298484802246, -1.9894174337387085, -2.0516977310180664, -1.2319469451904297, -1.8383702039718628, -1.9633283615112305], [-1.9159021377563477, -1.7567603588104248, -1.3830958604812622, -1.740067958831787, -1.9456660747528076, -2.1983461380004883], [-2.105015754699707, -1.6942380666732788, -1.4462732076644897, -1.6935241222381592, -1.6980078220367432, -2.3853936195373535], [-2.1473894119262695, -1.81721699

### Run Testing

In [64]:
#test_model_name = "05_31_2023_08_04_classifier_model.pth"
#test_model_path = f"{folder}/{test_model_name}"

test_model = BertClassifier()
test_model = test_model.to(device)
test_model.load_state_dict(torch.load(model_path))
test(test_model, test_dataloader)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Output: [[0.09333331137895584, 0.12520906329154968, 0.20777179300785065, 0.1638442575931549, 0.14590585231781006, 0.2639356851577759]]
Prediction: [5], Truth: [4.0]
Test Accuracy:  0.000
Model Output: [[0.08576416969299316, 0.16399188339710236, 0.15843790769577026, 0.1422109454870224, 0.2515603005886078, 0.19803482294082642]]
Prediction: [4], Truth: [2.0]
Test Accuracy:  0.000
Model Output: [[0.08736410737037659, 0.16592255234718323, 0.23735585808753967, 0.12783962488174438, 0.18480640649795532, 0.19671142101287842]]
Prediction: [2], Truth: [2.0]
Test Accuracy:  0.004
Model Output: [[0.11069808155298233, 0.1618581861257553, 0.22616545855998993, 0.11788883805274963, 0.1832762360572815, 0.20011313259601593]]
Prediction: [2], Truth: [4.0]
Test Accuracy:  0.004
Model Output: [[0.08806455880403519, 0.17017216980457306, 0.2075120359659195, 0.15551038086414337, 0.14960619807243347, 0.229134663939476]]
Prediction: [5], Truth: [4.0]
Test Accuracy:  0.004
Model Output: [[0.1186549291014671